In [18]:
import os
import glob # extract path of each file
import pandas as pd # data preprocessing
import json # read json file
from functools import reduce

In [29]:
textfiles = glob.glob('../components/computer_vision/data/Final Data/labels/train/*.txt')
# replace \\ with /
replace_text = lambda x: x.replace('\\','/')
textfiles = list(map(replace_text,textfiles))

In [30]:
textfiles

['../components/computer_vision/data/Final Data/labels/train/0000a16e4b057580.txt',
 '../components/computer_vision/data/Final Data/labels/train/00013bb4b9ddfbab.txt',
 '../components/computer_vision/data/Final Data/labels/train/00020ebf74c4881c.txt',
 '../components/computer_vision/data/Final Data/labels/train/00030b70a3abfcea.txt',
 '../components/computer_vision/data/Final Data/labels/train/0004b1e36d81b877.txt',
 '../components/computer_vision/data/Final Data/labels/train/0004fca29aa55afd.txt',
 '../components/computer_vision/data/Final Data/labels/train/0005817f5c4ef853.txt',
 '../components/computer_vision/data/Final Data/labels/train/00059157cde838d2.txt',
 '../components/computer_vision/data/Final Data/labels/train/0006e79d467197b9.txt',
 '../components/computer_vision/data/Final Data/labels/train/000784072165394b.txt',
 '../components/computer_vision/data/Final Data/labels/train/0007f2c11800286e.txt',
 '../components/computer_vision/data/Final Data/labels/train/000890d4d5c566a

In [53]:
import os

# step-2: read YOLO txt files
# from each txt file we need to extract
# filename, class_id, x_center, y_center, width, height
def extract_yolo_txt(filename):
    parser = []
    image_name = os.path.splitext(os.path.basename(filename))[0]  # file name without extension
    
    with open(filename, "r") as f:
        for line in f.readlines():
            values = line.strip().split()
            if len(values) != 5:
                continue  # skip malformed lines
            class_id, x_center, y_center, w, h = values
            parser.append([filename, class_id, x_center, y_center, w, h])
    
    return parser

In [71]:
parser_all = list(map(extract_yolo_txt,textfiles))

In [72]:
data = reduce(lambda x, y : x+y,parser_all)

In [73]:
df = pd.DataFrame(data,columns = ['filename', 'class_id', 'x_center', 'y_center', 'w', 'h'])

In [74]:
df.head()

,filename,class_id,x_center,y_center,w,h
0,../components/computer_vision/data/Final Data/...,6,0.091875,0.214583,0.183750,0.135833
1,../components/computer_vision/data/Final Data/...,6,0.091875,0.111667,0.128750,0.173333
2,../components/computer_vision/data/Final Data/...,6,0.345000,0.127500,0.131250,0.113334
3,../components/computer_vision/data/Final Data/...,6,0.635625,0.137916,0.167500,0.179167
4,../components/computer_vision/data/Final Data/...,6,0.745000,0.498333,0.352500,0.471667


In [75]:
df.shape

(36590, 6)

In [76]:
df['class_id'].value_counts()

class_id
3     1753
12    1554
2     1462
25    1316
36    1208
43    1205
9     1157
18    1085
19    1068
0     1006
44     945
27     938
46     937
42     926
10     922
41     916
30     884
7      852
8      816
34     770
15     760
11     756
22     753
14     751
16     746
1      637
33     627
6      621
28     619
4      616
31     604
21     600
24     594
13     594
23     569
38     551
26     549
49     470
48     441
40     433
32     426
17     422
5      421
35     411
47     369
37     318
29     212
Name: count, dtype: int64

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36590 entries, 0 to 36589
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  36590 non-null  object
 1   class_id  36590 non-null  object
 2   x_center  36590 non-null  object
 3   y_center  36590 non-null  object
 4   w         36590 non-null  object
 5   h         36590 non-null  object
dtypes: object(6)
memory usage: 1.7+ MB


In [78]:
cols = ['x_center', 'y_center', 'w', 'h']
df[cols] = df[cols].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36590 entries, 0 to 36589
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  36590 non-null  object 
 1   class_id  36590 non-null  object 
 2   x_center  36590 non-null  float64
 3   y_center  36590 non-null  float64
 4   w         36590 non-null  float64
 5   h         36590 non-null  float64
dtypes: float64(4), object(2)
memory usage: 1.7+ MB


In [79]:
images = df['filename'].unique()

In [80]:
len(images)

12686

In [81]:
# 80% train and 20% test
img_df = pd.DataFrame(images,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename']) # shuffle and pick 80% of images

In [82]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename']) # take rest 20% images

In [83]:
len(img_train), len(img_test)

(10149, 2537)

In [84]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [85]:
train_df.head()

,filename,class_id,x_center,y_center,w,h
7,../components/computer_vision/data/Final Data/...,13,0.380000,0.708438,0.246666,0.223125
8,../components/computer_vision/data/Final Data/...,13,0.690000,0.681875,0.618334,0.432500
9,../components/computer_vision/data/Final Data/...,14,0.106666,0.498125,0.168333,0.307500
10,../components/computer_vision/data/Final Data/...,14,0.245417,0.442812,0.172500,0.280625
11,../components/computer_vision/data/Final Data/...,7,0.879375,0.587769,0.198750,0.271709


In [86]:
test_df.head()

,filename,class_id,x_center,y_center,w,h
0,../components/computer_vision/data/Final Data/...,6,0.091875,0.214583,0.18375,0.135833
1,../components/computer_vision/data/Final Data/...,6,0.091875,0.111667,0.12875,0.173333
2,../components/computer_vision/data/Final Data/...,6,0.345000,0.127500,0.13125,0.113334
3,../components/computer_vision/data/Final Data/...,6,0.635625,0.137916,0.16750,0.179167
4,../components/computer_vision/data/Final Data/...,6,0.745000,0.498333,0.35250,0.471667


In [87]:
import os
from shutil import move

In [90]:
train_folder = '../components/computer_vision/data/Final Data/images/train'
test_folder = '../components/computer_vision/data/Final Data/images/test'
label_train_folder = '../components/computer_vision/data/Final Data/labels/train'
label_test_folder = '../components/computer_vision/data/Final Data/labels/test'

os.makedirs(test_folder, exist_ok=True)
os.makedirs(label_test_folder, exist_ok=True)

In [92]:
cols = ['filename', 'class_id', 'x_center', 'y_center', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [99]:
# Show full column content without truncation
pd.set_option("display.max_colwidth", None)

# Show all columns in the row (not cut off)
pd.set_option("display.max_columns", None)

# Show wider console output
pd.set_option("display.width", 0)

In [102]:
groupby_obj_test.head(5)

,filename,class_id,x_center,y_center,w,h
0,../components/computer_vision/data/Final Data/labels/train/0000a16e4b057580.txt,6,0.091875,0.214583,0.183750,0.135833
1,../components/computer_vision/data/Final Data/labels/train/0000a16e4b057580.txt,6,0.091875,0.111667,0.128750,0.173333
2,../components/computer_vision/data/Final Data/labels/train/0000a16e4b057580.txt,6,0.345000,0.127500,0.131250,0.113334
3,../components/computer_vision/data/Final Data/labels/train/0000a16e4b057580.txt,6,0.635625,0.137916,0.167500,0.179167
4,../components/computer_vision/data/Final Data/labels/train/0000a16e4b057580.txt,6,0.745000,0.498333,0.352500,0.471667
...,...,...,...,...,...,...
36583,../components/computer_vision/data/Final Data/labels/train/fff91551089c59dd.txt,41,0.415625,0.904166,0.085938,0.189583
36584,../components/computer_vision/data/Final Data/labels/train/fff91551089c59dd.txt,41,0.464063,0.919271,0.090625,0.157292
36585,../components/computer_vision/data/Final Data/labels/train/fff91551089c59dd.txt,41,0.591797,0.942188,0.088282,0.113541
36586,../components/computer_vision/data/Final Data/labels/train/fff91551089c59dd.txt,41,0.592187,0.824479,0.079687,0.226042


In [106]:
import os
import shutil
import pandas as pd

def save_labels(filename, dst_folder, group_obj):
    """
    Move only label (.txt) files from train -> test.
    Does not touch images.
    """
    base = os.path.basename(filename)  # e.g. "0000a16e4b057580.txt"

    # source label path (from train folder)
    src_txt = os.path.join("../components/computer_vision/data/Final Data/labels/train", base)

    # destination label path (into test folder)
    dst_txt = os.path.join(dst_folder, base)

    # make sure the destination exists
    os.makedirs(dst_folder, exist_ok=True)

    # move only the label file
    if os.path.exists(src_txt):
        shutil.move(src_txt, dst_txt)

In [107]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data, args=(test_folder, groupby_obj_test))

0       None
1       None
2       None
3       None
4       None
        ... 
2532    None
2533    None
2534    None
2535    None
2536    None
Length: 2537, dtype: object

In [112]:
import os

def save_yolo_labels(df, dst_folder):
    """
    Save YOLO labels from dataframe into .txt files.
    Expects columns: ['filename','class_id','x_center','y_center','w','h']
    """
    os.makedirs(dst_folder, exist_ok=True)

    # group by filename so each image gets its own .txt
    grouped = df.groupby('filename')

    for fname, group in grouped:
        # filename may be a path, we only need the base name without extension
        base = os.path.splitext(os.path.basename(fname))[0] + ".txt"
        out_path = os.path.join(dst_folder, base)

        # write YOLO format: class x_center y_center w h
        group[['class_id','x_center','y_center','w','h']].to_csv(
            out_path, sep=" ", index=False, header=False
        )

# --- regenerate train/test labels ---
train_labels_folder = "../components/computer_vision/data/Final Data/labels/train"
test_labels_folder  = "../components/computer_vision/data/Final Data/labels/test"

save_yolo_labels(train_df, train_labels_folder)
save_yolo_labels(test_df, test_labels_folder)
